In [1]:
# src/run_detect_track_save.py
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
from pathlib import Path
import time

# ---------- USER CONFIG ----------
VIDEO_IN = "video1.MP4"
OUT_DIR = Path("outputs/run1")
OUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL = "yolov8m.pt"        # try yolov8m or yolov8l for better accuracy
CONF_THRESH = 0.5           # raise to 0.6/0.7 to reduce false positives
IMG_SIZE = 1280             # larger helps small objects
SAVE_VIDEO = True
VIDEO_OUT = OUT_DIR / "annotated_camA.mp4"
SAVE_CSV = OUT_DIR / "tracks.csv"
TARGET_COCO_CLASSES = [2, 5, 7]  # COCO ids for car(2), bus(5), truck(7)
# ---------------------------------

model = YOLO(MODEL)  # automatically loads model (CPU or GPU if available)

cap = cv2.VideoCapture(VIDEO_IN)
fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(str(VIDEO_OUT), fourcc, fps, (w, h)) if SAVE_VIDEO else None

records = []  # will store frame detections for CSV

frame_id = 0
start = time.time()
while True:
    ok, frame = cap.read()
    if not ok:
        break

    # run detection on frame (returns Ultralyics Results)
    # Using model.predict allows fine control; model.track is also available if you want internal tracker
    results = model.predict(source=[frame],
                            imgsz=IMG_SIZE,
                            conf=CONF_THRESH,
                            verbose=False,
                            half=False)  # return list-like

    res = results[0]  # single image result
    boxes = res.boxes  # Boxes object

    # boxes.xyxy, boxes.conf, boxes.cls (torch tensors)
    if boxes is not None and len(boxes) > 0:
        xyxy = boxes.xyxy.cpu().numpy()  # Nx4
        confs = boxes.conf.cpu().numpy()  # N
        clss = boxes.cls.cpu().numpy().astype(int)  # N

        # filter classes and confidence
        keep_mask = [(c in TARGET_COCO_CLASSES) for c in clss]
        keep_mask = np.array(keep_mask) & (confs >= CONF_THRESH)

        xyxy = xyxy[keep_mask]
        confs = confs[keep_mask]
        clss = clss[keep_mask]

        for i, (box, conf, clsid) in enumerate(zip(xyxy, confs, clss)):
            x1, y1, x2, y2 = map(int, box)
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            # annotate frame
            label = f"{clsid}:{conf:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (10, 200, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

            # store record for CSV
            records.append({
                "frame": int(frame_id),
                "x1": int(x1), "y1": int(y1), "x2": int(x2), "y2": int(y2),
                "cx": int(cx), "cy": int(cy),
                "conf": float(conf), "class_id": int(clsid)
            })

    # write frame to output file (if requested)
    if SAVE_VIDEO:
        writer.write(frame)

    # optionally show (comment out if running headless)
    cv2.imshow("annot", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

    frame_id += 1

cap.release()
if writer:
    writer.release()
cv2.destroyAllWindows()
end = time.time()
print(f"Processed {frame_id} frames in {end-start:.1f}s")
pd.DataFrame(records).to_csv(SAVE_CSV, index=False)
print("Saved CSV:", SAVE_CSV)
print("Saved video:", VIDEO_OUT)


KeyboardInterrupt: 

In [1]:
import cv2
import os

# === Configuration ===
video_path = "video1.MP4"   # change this to your video file
output_folder = "frames_every_4s"
interval = 4  # seconds between frames

# === Create output directory ===
os.makedirs(output_folder, exist_ok=True)

# === Open video ===
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Cannot open video {video_path}")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count / fps
print(f"Video info: {frame_count} frames, {fps:.2f} fps, {duration:.2f}s total duration")

# === Frame extraction loop ===
count = 0
sec = 0

while sec < duration:
    cap.set(cv2.CAP_PROP_POS_MSEC, sec * 1000)  # move to the exact second mark
    success, frame = cap.read()
    if not success:
        break

    frame_name = os.path.join(output_folder, f"frame_{count:04d}.jpg")
    cv2.imwrite(frame_name, frame)  # save as-is (no resize)
    print(f"Saved: {frame_name}")

    count += 1
    sec += interval

cap.release()
print("✅ Done! All frames saved at original resolution.")


Video info: 10030 frames, 23.98 fps, 418.33s total duration
Saved: frames_every_4s\frame_0000.jpg
Saved: frames_every_4s\frame_0001.jpg
Saved: frames_every_4s\frame_0002.jpg
Saved: frames_every_4s\frame_0003.jpg
Saved: frames_every_4s\frame_0004.jpg
Saved: frames_every_4s\frame_0005.jpg
Saved: frames_every_4s\frame_0006.jpg
Saved: frames_every_4s\frame_0007.jpg
Saved: frames_every_4s\frame_0008.jpg
Saved: frames_every_4s\frame_0009.jpg
Saved: frames_every_4s\frame_0010.jpg
Saved: frames_every_4s\frame_0011.jpg
Saved: frames_every_4s\frame_0012.jpg
Saved: frames_every_4s\frame_0013.jpg
Saved: frames_every_4s\frame_0014.jpg
Saved: frames_every_4s\frame_0015.jpg
Saved: frames_every_4s\frame_0016.jpg
Saved: frames_every_4s\frame_0017.jpg
Saved: frames_every_4s\frame_0018.jpg
Saved: frames_every_4s\frame_0019.jpg
Saved: frames_every_4s\frame_0020.jpg
Saved: frames_every_4s\frame_0021.jpg
Saved: frames_every_4s\frame_0022.jpg
Saved: frames_every_4s\frame_0023.jpg
Saved: frames_every_4s\frame

In [2]:
from ultralytics import YOLO

# Load a pre-trained model (v8n = nano, fast; v8s = small, more accurate)
model = YOLO("yolov8n.pt")

# Train on your custom dataset
model.train(
    data="your_dataset/data.yaml",  # path to your data.yaml
    epochs=100,                     # number of epochs (increase if needed)
    imgsz=640,                       # image size (adjust if GPU allows)
    batch=16,                        # batch size
    name="vehicle_detection_custom", # run name
    project="project_train"          # folder where outputs are saved
)


Ultralytics 8.3.218  Python-3.11.2 torch-2.9.0+cpu CPU (AMD Ryzen 7 4800H with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=your_dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=vehicle_detection_custom, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspecti

RuntimeError: Dataset 'your_dataset/data.yaml' error  'your_dataset/data.yaml' does not exist